In [ ]:
import numpy as np
import pandas as pd
import copy
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

In [ ]:
df_kf = pd.read_csv('algoActual.csv')
df_kf

,Date,ClientName,Side,Symbol,QuoteID,ClOrdID,OrderID,TransactTime,OrderQty,OrdType,Price,AvgPx,CumQty,LeavesQty,OrdStatus,Fee,FilledValue,OtherFee,AlgoNote,BasketID
0,8/3/2021,1,B,600000.SH,0,20806333333000005,5000093,91533794,100,Limit\t,8.23,0.00,0,0,Canceled\t,0,0.0,0.000000,1,3
1,8/3/2021,1,B,600000.SH,0,20806333333000006,5000000,91533803,100,Limit\t,8.23,0.00,0,0,Canceled\t,0,0.0,0.000000,1,3
2,8/3/2021,1,B,000001.SZ,0,20806333333000009,4999907,91533826,100,Limit\t,16.22,0.00,0,0,Canceled\t,0,0.0,0.000000,1,3
3,8/3/2021,1,B,000001.SZ,0,20806333333000010,4999814,91533832,100,Limit\t,16.22,0.00,0,0,Canceled\t,0,0.0,0.000000,1,3
4,8/3/2021,1,S,603822.SH,20806334099000470,5230803134320000007,4999721,93200008,100,Limit\t,40.95,40.95,100,0,Filled\t,0,4095.0,4.458227,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11556,8/3/2021,1,S,002048.SZ,20806350256000073,5230803153644000015,3925385,145404623,100,Limit\t,20.01,20.01,100,0,Filled\t,0,2001.0,2.178489,1,3
11557,8/3/2021,1,B,002191.SZ,20806350256000101,5230803153644000021,3925292,145404647,100,Limit\t,10.16,10.15,100,0,Filled\t,0,1015.0,0.090031,1,3
11558,8/3/2021,1,S,002026.SZ,20806350256000068,5230803153644000022,3925199,145404647,100,Limit\t,12.61,12.61,100,0,Filled\t,0,1261.0,1.372851,1,3
11559,8/3/2021,1,S,603519.SH,20806350256000462,5230803153647000006,3925106,145407724,100,Limit\t,14.86,14.86,100,0,Filled\t,0,1486.0,1.617808,1,3


In [ ]:
df_kf['OrdStatus'] = df_kf['OrdStatus'].str.replace(r'\t', '')
df_kf['OrdType'] = df_kf['OrdType'].str.replace(r'\t', '')
df_kf

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


,Date,ClientName,Side,Symbol,QuoteID,ClOrdID,OrderID,TransactTime,OrderQty,OrdType,Price,AvgPx,CumQty,LeavesQty,OrdStatus,Fee,FilledValue,OtherFee,AlgoNote,BasketID
0,8/3/2021,1,B,600000.SH,0,20806333333000005,5000093,91533794,100,Limit,8.23,0.00,0,0,Canceled,0,0.0,0.000000,1,3
1,8/3/2021,1,B,600000.SH,0,20806333333000006,5000000,91533803,100,Limit,8.23,0.00,0,0,Canceled,0,0.0,0.000000,1,3
2,8/3/2021,1,B,000001.SZ,0,20806333333000009,4999907,91533826,100,Limit,16.22,0.00,0,0,Canceled,0,0.0,0.000000,1,3
3,8/3/2021,1,B,000001.SZ,0,20806333333000010,4999814,91533832,100,Limit,16.22,0.00,0,0,Canceled,0,0.0,0.000000,1,3
4,8/3/2021,1,S,603822.SH,20806334099000470,5230803134320000007,4999721,93200008,100,Limit,40.95,40.95,100,0,Filled,0,4095.0,4.458227,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11556,8/3/2021,1,S,002048.SZ,20806350256000073,5230803153644000015,3925385,145404623,100,Limit,20.01,20.01,100,0,Filled,0,2001.0,2.178489,1,3
11557,8/3/2021,1,B,002191.SZ,20806350256000101,5230803153644000021,3925292,145404647,100,Limit,10.16,10.15,100,0,Filled,0,1015.0,0.090031,1,3
11558,8/3/2021,1,S,002026.SZ,20806350256000068,5230803153644000022,3925199,145404647,100,Limit,12.61,12.61,100,0,Filled,0,1261.0,1.372851,1,3
11559,8/3/2021,1,S,603519.SH,20806350256000462,5230803153647000006,3925106,145407724,100,Limit,14.86,14.86,100,0,Filled,0,1486.0,1.617808,1,3


In [ ]:
n = df_kf.nunique(axis=0)
print("No.of.unique values in each column :\n",n)

No.of.unique values in each column :
 Date                1
ClientName          1
Side                2
Symbol           1052
QuoteID          2696
ClOrdID         11561
OrderID         11561
TransactTime    11383
OrderQty           91
OrdType             1
Price            4153
AvgPx            3590
CumQty             92
LeavesQty           1
OrdStatus           4
Fee                 1
FilledValue      4538
OtherFee         5709
AlgoNote            1
BasketID            1
dtype: int64


#### Q1: 每个股票的买/卖股票数量，金额，盈亏和收益率

In [ ]:
def Q1(table):
  df = copy.deepcopy(table)
  stock_list = df["Symbol"].unique()
  column_names = ["B_AMT", "S_AMT", "CASH_AMT", "P&L", "Return(%)"]
  temp_table = df[(df["OrdStatus"] == "Filled") | (df["OrdStatus"] == "New")]
  df_final = pd.DataFrame(index = stock_list, columns = column_names)

  # 计算每个股票的买/卖股票数量，金额，盈亏和收益率
  for stock in stock_list:
    stock_table = temp_table[temp_table["Symbol"] == stock]

    if stock_table.shape[0] == 0:
      df_final.loc[stock] = [0, 0, 0, 0, 0]
    
    elif stock_table.shape[0] == 1:
      if stock_table["Side"].values[0] == "B":
        df_final.loc[stock] = [stock_table[stock_table["Side"] == "B"]["OrderQty"].values[0], 
                               0,
                               stock_table["FilledValue"].values[0], 
                               0, 
                               0]
      else:
        df_final.loc[stock] = [0, 
                               stock_table[stock_table["Side"] == "S"]["OrderQty"].values[0], 
                               stock_table["FilledValue"].values[0], 
                               0, 
                               0]

    else:
      B_amount = stock_table[stock_table["Side"] == "B"]["OrderQty"].sum()
      S_amount = stock_table[stock_table["Side"] == "S"]["OrderQty"].sum()
      C_amount = stock_table["FilledValue"].sum()
      P_L = stock_table[stock_table["Side"] == "S"]["FilledValue"].sum() - stock_table[stock_table["Side"] == "B"]["FilledValue"].sum()
      RTN = ((stock_table["Price"].tolist()[-1] - stock_table["Price"].tolist()[0]) / (stock_table["Price"].tolist()[-1]))
      RTN = round(RTN,5)*100
      df_final.loc[stock] = [B_amount, S_amount, C_amount, P_L, RTN]
    
  return df_final

In [ ]:
df_Q1 = Q1(df_kf)
df_Q1

,B_AMT,S_AMT,CASH_AMT,P&L,Return(%)
600000.SH,0,0,0,0,0
000001.SZ,0,6100,108998.0,108998.0,1.056
603822.SH,200,200,16179.0,151.0,-2.018
601106.SH,5100,0,18598.0,-18598.0,1.897
600699.SH,500,500,25949.0,525.0,0.663
...,...,...,...,...,...
688199.SH,200,0,9878.0,0,0
688268.SH,200,0,17298.0,0,0
300685.SZ,0,100,8725.0,0,0
300782.SZ,100,0,39209.0,0,0


#### Q2: 每个股票总下单数， 撤单和成交的比例

In [ ]:
def Q2(table):
  df = copy.deepcopy(table)
  stock_list = df["Symbol"].unique()
  column_names = ["Total_Order", "Total_WithDraw", "Compelete_Ratio"]
  df_final_Q2 = pd.DataFrame(index = stock_list, columns = column_names)

  # 每个股票总下单数， 撤单和成交的比例
  for stock in stock_list:
    stock_table = df[df["Symbol"] == stock]
    order_number = stock_table["CumQty"].sum()  # 计算每只股票的下单数
    withdraw_number = stock_table[(stock_table["OrdStatus"] == "Canceled") | (stock_table["OrdStatus"] == "Rejected")]["CumQty"].sum()
    ratio = 1 - (withdraw_number/order_number)
    df_final_Q2.loc[stock] = [order_number, withdraw_number, ratio]
    df_final_Q2 = df_final_Q2.fillna(0)
    
  return df_final_Q2

In [ ]:
df_Q2 = Q2(df_kf)
df_Q2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in long_scalars
  if sys.path[0] == '':


,Total_Order,Total_WithDraw,Compelete_Ratio
600000.SH,0,0,0.0
000001.SZ,6100,0,1.0
603822.SH,400,0,1.0
601106.SH,5100,0,1.0
600699.SH,1000,0,1.0
...,...,...,...
688199.SH,200,0,1.0
688268.SH,200,0,1.0
300685.SZ,100,0,1.0
300782.SZ,100,0,1.0


#### Q3 每个股票交易费用，占盈亏的比例是多少？

In [ ]:
def Q3(table):
  df = copy.deepcopy(table)
  stock_list = df["Symbol"].unique()
  column_names = ["Transaction_Fee", "P&L","占盈亏比例(%)"]
  df_final_Q3 = pd.DataFrame(index = stock_list, columns = column_names)

  # 计算每支股票的交易费用和占盈亏的比例
  for stock in stock_list:
    stock_table = df[df["Symbol"] == stock]
    Total_fee = stock_table["OtherFee"].sum()  # 计算每只股票的总交易费用
    P_L = stock_table[stock_table["Side"] == "S"]["FilledValue"].sum() - stock_table[stock_table["Side"] == "B"]["FilledValue"].sum() # 计算每只股票的盈亏
    ratio = round(Total_fee/abs(P_L)*100,5) 
    df_final_Q3.loc[stock] = [Total_fee, P_L, ratio]
    df_final_Q3 = df_final_Q3.fillna(0)
    
  return df_final_Q3

In [ ]:
df_Q3 = Q3(df_kf)
df_Q3

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':


,Transaction_Fee,P&L,占盈亏比例(%)
600000.SH,0.000000,0.0,0.00000
000001.SZ,118.666125,108998.0,0.10887
603822.SH,9.600078,151.0,6.35767
601106.SH,1.649643,-18598.0,0.00887
600699.SH,15.538676,525.0,2.95975
...,...,...,...
688199.SH,0.876179,-9878.0,0.00887
688268.SH,1.534333,-17298.0,0.00887
300685.SZ,9.498907,8725.0,0.10887
300782.SZ,3.477838,-39209.0,0.00887


#### Q4 是否有没有平掉的仓位？如有， 是那个股票？ 多少股？

In [ ]:
def Q4(table):
  df = copy.deepcopy(table)
  stock_list = df["Symbol"].unique()
  column_names = ["Closed", "Stock_left"]
  temp_table = df[(df["OrdStatus"] == "Filled") | (df["OrdStatus"] == "New")]
  df_final_Q4 = pd.DataFrame(index = stock_list, columns = column_names)

  # 检查是否有尚未平仓的股票及其仓位
  for stock in stock_list:
    stock_table = temp_table[temp_table["Symbol"] == stock]

    if stock_table.shape[0] <= 1:
      df_final_Q4.loc[stock] = ["N", stock_table["OrderQty"].sum()]

    else:
      B_amount = stock_table[stock_table["Side"] == "B"]["OrderQty"].sum()
      S_amount = stock_table[stock_table["Side"] == "S"]["OrderQty"].sum()
      diff = B_amount - S_amount
      
      if diff == 0:
        df_final_Q4.loc[stock] = ["Y", 0]

      else:
        df_final_Q4.loc[stock] = ["N", diff]
    
  return df_final_Q4

In [ ]:
df_Q4 = Q4(df_kf)
df_Q4

,Closed,Stock_left
600000.SH,N,0
000001.SZ,N,-6100
603822.SH,Y,0
601106.SH,N,5100
600699.SH,Y,0
...,...,...
688199.SH,N,200
688268.SH,N,200
300685.SZ,N,100
300782.SZ,N,100


#### Q5:所有的股票，总的交易的金额， 按时间分布， 每10分钟的成交金额和收益是如何？

In [ ]:
def Q5(table):
  df = copy.deepcopy(table)

  # 转换时间到分钟
  df["TransactTime"] = df["TransactTime"].astype(str)
  df['hours'] = df["TransactTime"].str[:-7]
  df['minutes'] = df["TransactTime"].str[-7:-5]
  df['seconds'] = df["TransactTime"].str[-5:-3]
  df['milseconds'] = df["TransactTime"].str[-3:]
  df["time"] = (df['milseconds'].astype(int)/1000 + df['seconds'].astype(int))/60 + df['minutes'].astype(int) + df['hours'].astype(int)*60

  temp_table = df[(df["OrdStatus"] == "Filled") | (df["OrdStatus"] == "New")]
  stock_list = temp_table["Symbol"].unique()
  column_names = ["time", "trading volume", "Return"]
  time_span = df["time"].tolist()
  df_final_Q5 = pd.DataFrame(index = time_span, columns = column_names)
  print(time_span)

  for time in range(round(time_span[0]), round(time_span[-1]), 10):
    stock_table = temp_table[(temp_table["time"] >= time) & (temp_table["time"] <= (time + 10))]
    TraVol = stock_table["FilledValue"].sum()
    B_value = stock_table[stock_table["Side"] == "B"]["FilledValue"].sum()
    S_value = stock_table[stock_table["Side"] == "S"]["FilledValue"].sum()
    RTN = S_value - B_value
    rate = RTN/B_value
    df_final_Q5.loc[time] = [time, TraVol, rate]
  
  df_final_Q5 = df_final_Q5.dropna()

  return df_final_Q5

In [ ]:
df_Q5 = Q5(df_kf)
df_Q5

[555.5632333333333, 555.5633833333334, 555.5637666666667, 555.5638666666666, 572.0001333333333, 572.00015, 572.0005333333334, 572.0008833333334, 572.0026166666667, 572.00265, 572.0029666666667, 572.003, 572.0030333333333, 572.0034166666667, 572.0038, 572.0038166666667, 572.0038333333333, 572.0041833333333, 572.0042, 572.0056333333333, 572.00705, 572.0070666666667, 572.0071, 572.0071166666667, 572.00715, 572.0075, 572.0078666666667, 572.00895, 572.0093166666667, 572.0100333333334, 572.01005, 572.01045, 572.0104833333334, 572.01085, 572.0108666666666, 572.01125, 572.0117166666666, 572.0117166666666, 572.01175, 572.0118, 572.0122333333334, 572.0122666666666, 572.01275, 572.0137, 572.01375, 572.01455, 572.0149833333334, 572.0150166666667, 572.0154, 572.01545, 572.0154666666667, 572.0167833333334, 572.0175333333333, 572.01755, 572.0180666666666, 572.0182666666667, 572.019, 572.019, 572.0190833333334, 572.01915, 572.01915, 572.0191833333333, 572.01925, 572.0192833333333, 572.0193166666667, 5

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in double_scalars


,time,trading volume,Return
566.0,566.0,3059381.90,0.064602
576.0,576.0,1127535.65,0.147990
586.0,586.0,505874.00,-0.109094
596.0,596.0,16881566.63,-0.057901
606.0,606.0,2755576.42,0.642453
616.0,616.0,275864.00,2.507655
626.0,626.0,3328437.90,-0.166597
636.0,636.0,1228593.00,0.083714
646.0,646.0,573357.68,0.066288
656.0,656.0,1775069.00,0.084006


#### Q6：所有的股票，成交价格在5块钱以下的， 收益， 收益率， 赚钱、亏钱的股票个数是多少， 赚钱/亏钱总额是多少？

In [ ]:
def Q6(table):
  df = copy.deepcopy(table)
  temp_table = df[(df["OrdStatus"] == "Filled") | (df["OrdStatus"] == "New")]
  temp_table = temp_table[temp_table["AvgPx"] <= 5]
  stock_list = temp_table["Symbol"].unique()
  column_names = ["total return", "Return(%)", "Postive_Return"]
  df_final_Q6 = pd.DataFrame(index = stock_list, columns = column_names)

  for stock in stock_list:
    stock_table = temp_table[temp_table["Symbol"] == stock]
    B_value = stock_table[stock_table["Side"] == "B"]["FilledValue"].sum()
    S_value = stock_table[stock_table["Side"] == "S"]["FilledValue"].sum()
    RTN = S_value - B_value
    rate = RTN/B_value
      
    if RTN > 0:
      df_final_Q6.loc[stock] = [RTN, rate, "Y"]

    elif RTN == 0:
      df_final_Q6.loc[stock] = [0, 0, 0]

    else:
      df_final_Q6.loc[stock] = [RTN, rate, "N"]

  return df_final_Q6

In [ ]:
df_Q6 = Q6(df_kf)
print("赚钱的股票个数为： ", df_Q6[df_Q6["Postive_Return"] == "Y"].shape[0])
print("亏钱的股票个数为： ", df_Q6[df_Q6["Postive_Return"] == "N"].shape[0])
display(df_Q6)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in double_scalars
  


赚钱的股票个数为：  28
亏钱的股票个数为：  40


,total return,Return(%),Postive_Return
601106.SH,-18598.0,-1.0,N
600293.SH,-4458.0,-0.500505,N
000040.SZ,-5051.0,-0.691728,N
002002.SZ,-16449.0,-1.0,N
600715.SH,2374.0,0.998318,Y
...,...,...,...
000751.SZ,-12802.0,-1.0,N
000540.SZ,18185.0,inf,Y
000545.SZ,-17404.0,-1.0,N
002162.SZ,-1434.0,-1.0,N
